<a href="https://colab.research.google.com/github/chunkyshrapnel/Oregon_Project/blob/main/oregon_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import ee
import time

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()



To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=arKvzhmaEDKxkcyr_vHNXfMHas4VhVmW-zauhTAIubM&tc=GZ7GS4PQCwtvroVmw6x-qqCfBd-vjm5uC65V1o-CvsU&cc=NRcw2nKvIz3v2ehinZmNhNuj167WBCsdooG1PN6YnIo

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AZEOvhW_fKhS7LneSnwcRiN2OBSj01HeIAu9bCGs_maqVVGeAlq60DeydXI

Successfully saved authorization token.


In [ ]:
import ee
#ee.Authenticate()
ee.Initialize()

# select date
start_date = '2001-01-01';
end_date = '2001-03-01';

model = 'testy_8';
#image_coll = ee.ImageCollection('projects/openet/ensemble/conus/gridmet/monthly/provisional').filterDate(start_date, end_date)
image_coll = ee.ImageCollection('projects/openet/geesebal/conus/gridmet/monthly/provisional').filterDate(start_date, end_date)

states = ee.FeatureCollection('TIGER/2018/States');
oregon = states.filter(ee.Filter.eq('GEOID', '41'));
print(image_coll.first().projection().nominalScale().getInfo())

# Remove all images that are not in 10T or 11T
image_coll = image_coll.filter(ee.Filter.inList('mgrs_tile', ['10T','11T']))

# hacky list manipulation
to_list = image_coll.toList(image_coll.size());
size_of_sub_list = to_list.size().divide(2);

ten_t_list = to_list.splice(size_of_sub_list, size_of_sub_list)       # EPSG:32610 T10
eleven_t_list = to_list.splice(0, size_of_sub_list)                   # EPSG:32611 T11

# combine the two sepearte lists into one list of list pairs
combinations = ten_t_list.zip(eleven_t_list);

def mosaic_two_images(lst):

  lst = ee.List(lst);

  my_id = ee.Image(lst.get(0)).id().slice(4);

  # crs: EPSG:4326 (not real?)
  new_mosaic = ee.Image(ee.ImageCollection([lst.get(0), lst.get(1)]).mosaic());

  new_mosaic = new_mosaic.reproject(**{'crs': 'EPSG:32610', 'scale':30}).reduceResolution(**{
      'reducer': ee.Reducer.mean(),
      'maxPixels': 65536}).set("system:index",my_id);

  return new_mosaic;


mosaic_images = combinations.map(mosaic_two_images)


print(ee.ImageCollection(mosaic_images).first().projection().nominalScale().getInfo())


def export_tif(img):

  # clip so that idahoe and washington datat is left out
  img = ee.Image(img);
  img = img.clip(oregon);

  # convert all bands to unsigned int16. If types are not consistent than an error is thrown.
  img = img.toUint16();

  # Create file path
  path = 'oregon/' + model + '_' + start_date + '_' + end_date + '/' + img.id().getInfo()

  print(img.projection().getInfo())
  print(img.projection().nominalScale().getInfo())

  # Export to Google bucket as a .tif file. (GeoTIFF)
  task = ee.batch.Export.image.toCloudStorage(**{
    'image': img,
    #description: path,
    'bucket': 'openet_temp',
    'fileNamePrefix': path,
    'crs': img.projection(),
    #'crsTransform': [30, 0, 239985, 0, -30, 5299995],
    #'scale': 30,
    'region': oregon.geometry(),
     #'maxPixels': 1e13

  })

  task.start()

  while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    time.sleep(5)

# For loop to export each image in the list
for i in range(mosaic_images.length().getInfo()):

  export_tif(mosaic_images.get(i));





In [52]:
# This is a simplified version of the Oregon ET export scipt.
# It is simplified to higlight a bug.
# The bug happens when exporting images. Even though the export is function is essentially copied from the JS script(which works excelently),
# When messing with the arguments in the function: ee.batch.Export.image.toCloudStorage(), one of two undesirable outcomes can happen.
#
# Outcome 1: the image resolution is way smaller than it should be. There will be about 20 pixels total. The image size will be about 3 KB each.
# The correct image exports from the JS script are around 350 KB. You can see these images in the "testy" foulder.
#
# Outcome 2; The image resolution is way larger than it should be. One image had a pixel count of 310493636550. To revent the task manager from throwing
# errors, the 'maxPixel' length has to be set to 1e13. When these images are exported, it will take about 3 hours for the task to be finished.
# The JS images would eport in about 2 minutes. You can see the fratured images in the "testy_6" folder.
#
# Commenting out the 'scale' variable will lead to option 1. With the 'scale' vaible you will get option 2.
# Commenting out the 'scale' variable and changing the 'crs' vairble to 'EPSG:32610' will lead to option 2.
# Commenting out and changing the 'crs' varible to 'EPSG:4326' seems to have no effect
# The 'crsTransorm' vraible seems to have no effect. I also don't really understand this varible.
#
# Change the 'output_folder' varible to select where the images will be exported.
model = '100_scale'

import ee
#ee.Authenticate()
ee.Initialize()

# select date
start_date = '2001-01-01';
end_date = '2001-02-01';

# Pull ET data
image_coll = ee.ImageCollection('projects/openet/geesebal/conus/gridmet/monthly/provisional').filterDate(start_date, end_date)

# select boundary
states = ee.FeatureCollection('TIGER/2018/States');
oregon = states.filter(ee.Filter.eq('GEOID', '41'));

# Remove all images that are not in 10T or 11T
image_coll = image_coll.filter(ee.Filter.inList('mgrs_tile', ['10T','11T']))

# hacky list manipulation
to_list = image_coll.toList(image_coll.size());
size_of_sub_list = to_list.size().divide(2);

ten_t_list = to_list.splice(size_of_sub_list, size_of_sub_list)       # EPSG:32610 T10
eleven_t_list = to_list.splice(0, size_of_sub_list)                   # EPSG:32611 T11

# combine the two sepearte lists into one list of list pairs
combinations = ten_t_list.zip(eleven_t_list);

def mosaic_two_images(lst):

  lst = ee.List(lst);

  my_id = ee.Image(lst.get(0)).id().slice(4);

  # crs: EPSG:4326 (not real?)
  new_mosaic = ee.Image(ee.ImageCollection([lst.get(0), lst.get(1)]).mosaic()).set("system:index",my_id)

  '''new_mosaic = new_mosaic.reproject(**{'crs': 'EPSG:32610', 'scale':30}).reduceResolution(**{
      'reducer': ee.Reducer.mean(),
      'maxPixels': 65536}).set("system:index",my_id)'''

  return new_mosaic;


mosaic_images = combinations.map(mosaic_two_images)

def export_tif(img):

  # clip so that idahoe and washington datat is left out
  img = ee.Image(img);
  img = img.clip(oregon);

  # convert all bands to unsigned int16. If types are not consistent than an error is thrown.
  img = img.toUint16();

  # Create file path
  path = 'oregon/' + model + '_' + start_date + '_' + end_date + '/' + img.id().getInfo()

  # prints projection and scale
  #print(img.projection().getInfo())
  #print(img.projection().nominalScale().getInfo())

  # Export to Google bucket as a .tif file. (GeoTIFF)
  task = ee.batch.Export.image.toCloudStorage(**{
    'image': img,
    #description: path,
    'bucket': 'openet_temp',
    'fileNamePrefix': path,
    'crs': img.projection(),
    #'crsTransform': [30, 0, 239985, 0, -30, 5299995],
    'scale': 100,
    'region': oregon.geometry(),
     #'maxPixels': 1e13

  })

  task.start()

  while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    time.sleep(5)

# For loop to export each image in the list
for i in range(mosaic_images.length().getInfo()):

  export_tif(mosaic_images.get(i));


Polling for task (id: NA5U7KMSTU5YJH3EIURWZI37).
Polling for task (id: NA5U7KMSTU5YJH3EIURWZI37).
Polling for task (id: NA5U7KMSTU5YJH3EIURWZI37).
Polling for task (id: NA5U7KMSTU5YJH3EIURWZI37).
Polling for task (id: NA5U7KMSTU5YJH3EIURWZI37).
Polling for task (id: NA5U7KMSTU5YJH3EIURWZI37).
Polling for task (id: NA5U7KMSTU5YJH3EIURWZI37).
Polling for task (id: NA5U7KMSTU5YJH3EIURWZI37).
Polling for task (id: NA5U7KMSTU5YJH3EIURWZI37).
Polling for task (id: NA5U7KMSTU5YJH3EIURWZI37).
Polling for task (id: NA5U7KMSTU5YJH3EIURWZI37).
Polling for task (id: NA5U7KMSTU5YJH3EIURWZI37).
Polling for task (id: NA5U7KMSTU5YJH3EIURWZI37).
Polling for task (id: NA5U7KMSTU5YJH3EIURWZI37).
Polling for task (id: NA5U7KMSTU5YJH3EIURWZI37).
Polling for task (id: NA5U7KMSTU5YJH3EIURWZI37).
Polling for task (id: NA5U7KMSTU5YJH3EIURWZI37).
Polling for task (id: NA5U7KMSTU5YJH3EIURWZI37).
Polling for task (id: NA5U7KMSTU5YJH3EIURWZI37).
Polling for task (id: NA5U7KMSTU5YJH3EIURWZI37).
Polling for task (id